https://zhuanlan.zhihu.com/p/36331115

&emsp;&emsp;比如有两个高斯分布, 其均值和方差分别为：$\mu_{1}, \mu_{2},\sigma_{1}, \sigma_{2}$。它的数据混合起来了，得到了一个整体的分布，`EM`算法就是从混合的这个数据中获取$\mu_{1}, \mu_{2},\sigma_{1}, \sigma_{2}$。

## EM算法

- **三硬币模型**

&emsp;&emsp;有三枚硬币$A, B, C$, 它们抛出正面的概率分别是:$\pi, p, q$。$A$抛出正面的话，之后就去抛$B$，$A$抛出反面的话，之后就去抛$C$。如何依据观测到的数据，比如硬币的正反序列，反推$\pi, p, q$呢？于此相对的，这里还有一个不可观测数据，或者称之为隐变量$z$, 它表示$A$的结果。

&emsp;&emsp;比如说看到一个最终结果是正面，这个正面我们并不知道是由$A$抛出正面继而让$B$产生正面得到的，还是$A$抛出反面继而让$C$产生得到的。这个事情就叫做隐变量。

&emsp;&emsp;随机变量$z$(观测不到的数据)满足概率分布：$z \sim b(1, \pi)$。则依据隐变量产生的数据可以表示为：$(z_{1}, y_{1})(z_{2}, y_{2}),\cdots, (z_{N}, y_{N})$。

1. $z_{1}=1, y_{1} \sim b(1, p)$;
2. $z_{1}=0, y_{1} \sim b(1, q)$。

$$
\prod_{i=1}^{N} p(y_{i} | \theta) = \prod_{i=1}^{N}[\sum_{Z}p(y_{i}, z|\theta)] = \prod_{i=1}^{N}[\pi p^{y_{i}}(1-p)^{1-y_{i}} + (1-\pi)q^{y_{i}}(1-q)^{1-y_{i}}]
$$

&emsp;&emsp;上述公示由于是连乘，在计算机语言中，可能会造成下溢出。所以我们一般将其取个$ln$。

$$
p(y,z|\theta) = max(ln\prod_{i=1}^{N}p(y_{i},z_{i}|\theta))
$$

&emsp;&emsp;上述公示中由于存在隐变量，所以没办法用之前的求极大似然的方法对其进行直接计算。

### EM算法的引入

- **EM算法**

输入：观测变量数据$Y$, 隐变量数据$Z$, 联合分布$P(Y,Z|\theta)$, 条件分布$P(Z|Y, \theta)$。

输出：模型参数$\theta$ (三币问题中就是$\pi, p, q$)。

(1) 选择参数的初值$\theta^{0}$, 开始迭代；

(2) $E$步：记$\theta^{i}$为第$i$次迭代参数$\theta$的估计值，在第$i+1$次迭代的$E$步，计算：

$$
Q(\theta, \theta^{i}) = E_{Z}[logP(Y,Z|\theta) | Y, \theta^{i}]
$$

(3) $M$步：求使$Q(\theta, \theta^{i})$极大化的$\theta$, 确定第$i+1$次迭代的参数估计值$\theta^{(i+1)}$。

$$
\theta^{(i+1)} = arg\max_{\theta}Q(\theta, \theta^{(i)})
$$

(4) 重复第(2)步和第(3)步，直到收敛。

### EM算法的导出

&emsp;&emsp;我们是期望依据观测样本使得整个数据产生的概率最大：

$$
L(\theta)=\ln P(Y \mid \theta)=\ln \sum_{z} P(Y, Z \mid \theta)=\ln \left(\sum_{Z} P(Z \mid \theta) P(Y \mid Z, \theta)\right)
$$

&emsp;&emsp;如何来计算呢？我们让每次迭代更新之后，它的值都会变大就好：

$$
\begin{aligned}
L(\theta)-L\left(\theta^{(i)}\right) &=\ln \left(\sum_{z} P(Z \mid \theta) P(Y \mid Z, \theta)\right)-\ln P\left(Y \mid \theta^{(i)}\right) \\
&=\ln \left(\sum_{z} P\left(Z \mid Y, \theta^{(i)}\right) \frac{P(Z \mid \theta) P(Y \mid Z, \theta)}{P\left(Z \mid Y, \theta^{(i)}\right)}\right)-\ln P\left(Y \mid \theta^{(i)}\right) \\
& \geq \sum_{Z} P\left(Z \mid Y, \theta^{(i)}\right) \ln \left(\frac{P(Z \mid \theta) P(Y \mid Z, \theta)}{P\left(Z \mid Y, \theta^{(i)}\right)}\right)-\sum_{z} P\left(Z \mid Y, \theta^{(i)}\right) \ln P\left(Y \mid \theta^{(i)}\right) \\
&=\sum_{Z} P\left(Z \mid Y, \theta^{(i)}\right) \ln \left(\frac{P(Z \mid \theta) P(Y \mid Z, \theta)}{P\left(Z \mid Y, \theta^{(i)}\right) P\left(Y \mid \theta^{(i)}\right)}\right)
\end{aligned}
$$

&emsp;&emsp;上述公式的不等式推倒用到了`Jensen`不等式，定义如下：如果$f$是凸函数，$X$是随机变量，那么：$E[f(X)] \geq f(E[X])$。当且仅当$X$是常量时，该式取等号。其中，$E(X)$表示$X$的数学期望。$ln$为凹函数。

&emsp;&emsp;我们是期望下面这个式子越大越好：

$$
L(\theta)-L\left(\theta^{(i)}\right) = \sum_{Z} P\left(Z \mid Y, \theta^{(i)}\right) \ln \left(\frac{P(Z \mid \theta) P(Y \mid Z, \theta)}{P\left(Z \mid Y, \theta^{(i)}\right) P\left(Y \mid \theta^{(i)}\right)}\right)
$$

&emsp;&emsp;但是上述式子中$\theta^{(i)}$是固定不动的，我们需要调整的是$\theta$。

$$
L(\theta)-L\left(\theta^{(i)}\right) = \sum_{Z} P\left(Z \mid Y, \theta^{(i)}\right) \ln \left(P(Z \mid \theta) P(Y \mid Z, \theta)\right) = B(\theta, \theta^{(i)})
$$

&emsp;&emsp;求解: 

$$
\begin{aligned}
\theta^{(i+1)} &=\operatorname{argmax}_{\theta} B\left(\theta, \theta^{(i)}\right) \\
&=\operatorname{argmax}_{\theta} \sum_{Z} P\left(Z \mid Y, \theta^{(i)}\right)(\ln P(Z \mid \theta) P(Y \mid Z, \theta)) \\
&=\operatorname{argmax}_{\theta} \sum_{Z} P\left(Z \mid Y, \theta^{(i)}\right)(\ln P(Y, Z \mid \theta)) \\
&=\operatorname{argmax}_{\theta} Q\left(\theta, \theta^{(i)}\right)
\end{aligned}
$$

- 小结

1. `EM`算法通过迭代逐步近似极大化$L$;
2. 为了每次都能够极大化$L$, 需保证每个步骤中$L(\theta)-L(\theta^{(i)})$大于0。
3. 通过找到当前$L(\theta)-L(\theta^{(i)})$的下界，不断提高该下界。

### EM算法收敛性证明

## 高斯混合模型

## EM算法的推广

### F函数的极大-极大算法

### GEM算法